In [ ]:
!pip install -q xformers

In [ ]:
!pip install git+https://github.com/huggingface/diffusers

In [ ]:
from huggingface_hub import notebook_login

# Authenticate with Hugging Face
notebook_login()

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

In [ ]:
!pip install datasets accelerate peft diffusers bitsandbytes

In [ ]:
#!/usr/bin/env bash
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --dataset_name="your-dataset-name" \
  --output_dir="output-directory" \
  --caption_column="prompt" \
  --mixed_precision="bf16" \
  --instance_prompt="a highly detailed MMORPG asset" \
  --resolution=512 \
  --train_batch_size=32 \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --learning_rate=2e-4 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=50 \
  --max_train_steps=1000 \
  --checkpointing_steps=250 \
  --validation_prompt="a highly detailed MMORPG character portrait, fantasy race, intricate facial features, game art style, 4k resolution"\
  --enable_xformers_memory_efficient_attention \
  --dataloader_num_workers=16 \
  --rank=128 \
  --report_to="tensorboard" \
  --push_to_hub

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL, DPMSolverMultistepScheduler
import matplotlib.pyplot as plt
from IPython.display import display

# Load the fine-tuned model
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
pipe.load_lora_weights("charactergen_model_improved")
pipe = pipe.to("cuda")

# Enable attention slicing for memory efficiency
pipe.enable_attention_slicing()

# Use a more efficient scheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Function to generate and display an image
def generate_and_display_image(prompt, num_inference_steps=20):
    image = pipe(prompt=prompt, num_inference_steps=num_inference_steps).images[0]
    
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.axis('off')
    plt.title(prompt)
    display(plt.gcf())
    plt.close()

# Test prompts
test_prompts = [
    "a highly detailed MMORPG character asset of a noble elven warrior with intricate armor, long flowing hair, and a determined expression, 4k resolution, cinematic lighting",
    "an MMORPG character asset of a wise old wizard with a long beard and ornate robes, holding a magical staff",
    "an MMORPG character asset of a stealthy rogue assassin with dark leather armor and a hooded cloak",
    "an MMORPG character asset of a fierce orc chieftain with tribal markings and imposing battle gear"
]

# Generate and display images for each prompt
for prompt in test_prompts:
    print(f"Generating: {prompt}")
    generate_and_display_image(prompt)
    print("\n")

# Demonstrate batch generation
def get_inputs(batch_size=1):
    generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
    prompts = batch_size * [test_prompts[0]]  # Using the first prompt for this example
    num_inference_steps = 20

    return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps}

# Generate a batch of images
batch_size = 4  # Adjust based on your GPU capabilities
images = pipe(**get_inputs(batch_size=batch_size)).images

# Display the batch of images
plt.figure(figsize=(20, 20))
for i, image in enumerate(images):
    plt.subplot(2, 2, i+1)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Character {i+1}")
plt.tight_layout()
display(plt.gcf())
plt.close()

In [ ]:
!pip install matplotlib


In [ ]:
import shutil
from huggingface_hub import HfFolder

cache_path = HfFolder.cache_home
if os.path.exists(cache_path):
    shutil.rmtree(cache_path)
    print(f"Cleared Hugging Face cache at {cache_path}")

In [ ]:
!apt-get clean
!apt-get autoclean
!rm -rf /tmp/
!pip cache purge

In [ ]:
!find / -maxdepth 1 -type f -size +100M -exec ls -lh {} \; 2>/dev/null

In [ ]:
import os
import shutil
from huggingface_hub import HfFolder

# Get the cache directory
cache_path = HfFolder().cache_dir

if os.path.exists(cache_path):
    # List contents before deletion
    print("Contents of cache directory before deletion:")
    for item in os.listdir(cache_path):
        print(item)
    
    # Remove contents of the cache directory
    for item in os.listdir(cache_path):
        item_path = os.path.join(cache_path, item)
        if os.path.isfile(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
    
    print(f"\nCleared contents of Hugging Face cache at {cache_path}")
    
    # List contents after deletion
    print("\nContents of cache directory after deletion:")
    for item in os.listdir(cache_path):
        print(item)
else:
    print(f"Hugging Face cache directory not found at {cache_path}")

# Check available space
!df -h /